# Open Weather Map - API
### PyOWM - A Python wrapper around the OpenWeatherMap API   
### install: pip install pyowm

In [1]:
import datetime
import json

import pyowm

In [2]:
# get a handler using my api key
owm = pyowm.OWM('1dc779244c89ccffd3d0d0cda8f3a664')

---

In [3]:
# Search for current weather in London (Great Britain)
observation = owm.weather_at_place('Montreal,CA')
w = observation.get_weather()
print(w)

<pyowm.webapi25.weather.Weather - reference time=2018-11-07 20:43:00+00, status=rain, detailed status=light intensity shower rain>


In [4]:
# Weather details
w.get_wind()                  # {'speed': 4.6, 'deg': 330}
w.get_humidity()              # 87
print('hour = {}'.format(w.get_reference_time('date')))
print('temperature = {}'.format(w.get_temperature('celsius')['temp']))  # {'temp_max': 10.5, 'temp': 9.7, 'temp_min': 9.0}

hour = 2018-11-07 20:43:00+00:00
temperature = 8.65


---

# Forecast for the next 5 days, at every 3 hours

In [77]:
fcst = owm.three_hours_forecast('Montreal,CA')
for weather in fcst.get_forecast():
    date = weather.get_reference_time(timeformat='date') - datetime.timedelta(hours=4)
    print (date, weather.get_temperature('celsius')['temp'], weather.get_status(), weather.get_rain())

2018-09-21 17:00:00+00:00 25.44 Rain {'3h': 0.0050000000000008}
2018-09-21 20:00:00+00:00 22.26 Rain {'3h': 2.525}
2018-09-21 23:00:00+00:00 20.38 Rain {'3h': 9}
2018-09-22 02:00:00+00:00 18.0 Rain {'3h': 0.045000000000002}
2018-09-22 05:00:00+00:00 12.94 Rain {'3h': 0.31}
2018-09-22 08:00:00+00:00 10.61 Clear {}
2018-09-22 11:00:00+00:00 13.2 Clear {}
2018-09-22 14:00:00+00:00 15.16 Clear {}
2018-09-22 17:00:00+00:00 15.44 Clear {}
2018-09-22 20:00:00+00:00 10.91 Clear {}
2018-09-22 23:00:00+00:00 7.82 Clear {}
2018-09-23 02:00:00+00:00 6.1 Clear {}
2018-09-23 05:00:00+00:00 6.82 Rain {'3h': 0.004999999999999}
2018-09-23 08:00:00+00:00 8.71 Clouds {}
2018-09-23 11:00:00+00:00 13.5 Clouds {}
2018-09-23 14:00:00+00:00 15.46 Clouds {}
2018-09-23 17:00:00+00:00 14.35 Clouds {}
2018-09-23 20:00:00+00:00 10.59 Clouds {}
2018-09-23 23:00:00+00:00 6.1 Clear {}
2018-09-24 02:00:00+00:00 3.4 Clear {}
2018-09-24 05:00:00+00:00 1.92 Clouds {}
2018-09-24 08:00:00+00:00 2.81 Clear {}
2018-09-24 11:

# Extract info of the next 24 hours

In [14]:
result = {
    "date": datetime.datetime.now().strftime('%Y-%m-%d %H:%M'),
    "day_precipitation_mm": 0,
    "hourly_temp": {}
}

# get forecast of the next 5 days
fcst = owm.three_hours_forecast('Montreal,CA')
forecast = fcst.get_forecast()

# parse the next 24 hours (8 x 3 hours)
for i in range(8):
    weather = forecast.get(i)
    
    # get time and temperature for this weather forecast object
    temp = weather.get_temperature('celsius')['temp']
    date = weather.get_reference_time(timeformat='date') - datetime.timedelta(hours=4)    
        
    # compute total day precipitation
    precipitation_mm = 0
    status = weather.get_status()
    if status == 'Rain':
        precipitation_mm = weather.get_rain()['3h']
    if status == 'Snow':
        precipitation_mm = weather.get_snow()['3h']
    result["day_precipitation_mm"] += precipitation_mm

    # fill hourly temperature array
    h = date.hour
    result["hourly_temp"][h] = temp
    h += 1
    result["hourly_temp"][h if h <= 24 else (h - 24)] = temp
    h += 1    
    result["hourly_temp"][h if h <= 24 else (h - 24)] = temp
    
    print (date.hour, temp, status, precipitation_mm)
    
print('-------------------------------')
print(result)

with open('weather.json', 'w') as outfile:
    json.dump(result, outfile)

17 24.29 Rain 0.0050000000000008
20 21.5 Rain 2.525
23 20.02 Rain 9
2 18.04 Rain 0.045000000000002
5 12.94 Rain 0.31
8 10.61 Clear 0
11 13.2 Clear 0
14 15.16 Clear 0
-------------------------------
{'date': '2018-09-21 16:18', 'day_precipitation_mm': 11.885000000000003, 'hourly_temp': {17: 24.29, 18: 24.29, 19: 24.29, 20: 21.5, 21: 21.5, 22: 21.5, 23: 20.02, 24: 20.02, 1: 20.02, 2: 18.04, 3: 18.04, 4: 18.04, 5: 12.94, 6: 12.94, 7: 12.94, 8: 10.61, 9: 10.61, 10: 10.61, 11: 13.2, 12: 13.2, 13: 13.2, 14: 15.16, 15: 15.16, 16: 15.16}}


---